In [1]:
from __future__ import division
import sys
sys.path.insert(0, "~/.local/lib/python3.6/site-packages")

import sympy
from sympy import *
from sympy.physics.quantum import *

def express(a, b, name):
    sym = symbols(name)
    sol = solve(a-sym, b)
    assert len(sol) == 1
    return (sym, sol[0])


def frange(x, y, jump):
    i = 0
    curr = x + i*jump
    
    while curr < y:
        yield curr
        i += 1
        curr = x + i*jump
    

import gmpy2
import mpmath
gmpy2.get_context().precision = 10
mpmath.mp.dps = 10

import numpy
import scipy.linalg

In [4]:
s1_ = (1/sqrt(2)) * \
    Matrix([
        [0,1,0],
        [1,0,1],
        [0,1,0]
    ])
    
s2_ = (1/sqrt(2)) * \
    Matrix([
        [0,-I,0],
        [I,0,-I],
        [0, I,0]
    ])

s3_ = Matrix([
        [1,0, 0],
        [0,0, 0],
        [0,0,-1]
    ])

In [9]:
s1_*s2_

Matrix([
[I/2, 0, -I/2],
[  0, 0,    0],
[I/2, 0, -I/2]])

In [10]:
s2_*s1_

Matrix([
[-I/2, 0, -I/2],
[   0, 0,    0],
[ I/2, 0,  I/2]])

In [11]:
s2_*s3_

Matrix([
[          0, 0,           0],
[sqrt(2)*I/2, 0, sqrt(2)*I/2],
[          0, 0,           0]])

In [12]:
s3_*s2_

Matrix([
[0, -sqrt(2)*I/2, 0],
[0,            0, 0],
[0, -sqrt(2)*I/2, 0]])

In [13]:
s3_*s1_

Matrix([
[0,  sqrt(2)/2, 0],
[0,          0, 0],
[0, -sqrt(2)/2, 0]])

In [14]:
s1_*s3_

Matrix([
[        0, 0,          0],
[sqrt(2)/2, 0, -sqrt(2)/2],
[        0, 0,          0]])

In [17]:
s1_*s1_

Matrix([
[1/2, 0, 1/2],
[  0, 1,   0],
[1/2, 0, 1/2]])

In [23]:
s2_*s2_

Matrix([
[ 1/2, 0, -1/2],
[   0, 1,    0],
[-1/2, 0,  1/2]])

In [24]:
s3_*s3_

Matrix([
[1, 0, 0],
[0, 0, 0],
[0, 0, 1]])

In [29]:
n = 4
for a in range(0,n+1):
    for b in range(0,n-a+1):
        c = n-a-b
        print(a+b+c, a, b, c)

4 0 0 4
4 0 1 3
4 0 2 2
4 0 3 1
4 0 4 0
4 1 0 3
4 1 1 2
4 1 2 1
4 1 3 0
4 2 0 2
4 2 1 1
4 2 2 0
4 3 0 1
4 3 1 0
4 4 0 0
